## IMPORTING LIBRARIES

In [4]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras import regularizers
from sklearn.preprocessing import StandardScaler
from keras.callbacks import EarlyStopping

## IMPORTING DATASET

In [5]:
df = pd.read_csv('diabetes.csv')

In [6]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


No Null values.

In [8]:
#Independent features
X = df.drop('Outcome', 1)
# Dependent Features
y = df['Outcome']

In [9]:
# replacing any missing values
X.replace('?', -99999, inplace=True)

In [10]:
X.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33


In [11]:
y.head()

0    1
1    0
2    1
3    0
4    1
Name: Outcome, dtype: int64

In [12]:
#scaling input
a = StandardScaler()
a.fit(X)
x = a.transform(X)

In [13]:
# One hot encode
Y = []
for i in y:
  if i == 1:
    Y.append([1, 0])  # class 1
  elif i == 0:
    Y.append([0, 1])  # class 0

In [14]:
# Converting into numpy array
x = np.array(x)
Y = np.array(Y)
print(x.shape)
print(Y.shape)

(768, 8)
(768, 2)


In [15]:
# TRAIN tEST SPLIT USING SKLEARN
x_train, x_test, Y_train, Y_test = train_test_split(x, Y, test_size=0.2, random_state=0)
print(x_train.shape,x_test.shape,Y_train.shape,Y_test.shape)

(614, 8) (154, 8) (614, 2) (154, 2)


## L1 REGULARIZATION

In [16]:
# BUILDING NEURAL NETWORK
model_l1 = Sequential()
model_l1.add(Dense(500, input_dim=8, kernel_regularizer=regularizers.l1(1e-4), bias_regularizer=regularizers.l1(1e-4), activity_regularizer=regularizers.l1(1e-4), activation='sigmoid'))
model_l1.add(Dense(100,kernel_regularizer=regularizers.l1(1e-4), bias_regularizer=regularizers.l1(1e-4),activity_regularizer=regularizers.l1(1e-4), activation='sigmoid'))
model_l1.add(Dense(2, activation='softmax'))
model_l1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [39]:
model_l1.fit(x_train,Y_train, epochs=1000, batch_size=70,validation_data=(x_test, Y_test),verbose=0);

## L2 REGULARIZATION

In [18]:
# BUILDING NEURAL NETWORK
model_l2 = Sequential()
model_l2.add(Dense(500, input_dim=8, kernel_regularizer=regularizers.l2(1e-4), bias_regularizer=regularizers.l2(1e-4), activity_regularizer=regularizers.l2(1e-4), activation='sigmoid'))
model_l2.add(Dense(100,kernel_regularizer=regularizers.l2(1e-4), bias_regularizer=regularizers.l2(1e-4),activity_regularizer=regularizers.l1(1e-4), activation='sigmoid'))
model_l2.add(Dense(2, activation='softmax'))
model_l2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [37]:
model_l2.fit(x_train,Y_train, epochs=1000, batch_size=70, validation_data=(x_test, Y_test), verbose=0)

## Early Stopping

In [20]:
# BUILDING NEURAL NETWORK
model_es = Sequential()
model_es.add(Dense(500, input_dim=8, activation='sigmoid'))
model_es.add(Dense(100, activation='sigmoid'))
model_es.add(Dense(2, activation='softmax'))
model_es.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [40]:
es = EarlyStopping(monitor='val_loss',patience=100)
model_es.fit(x_train,Y_train, epochs=1000, batch_size=70, validation_data=(x_test,Y_test),callbacks=[es],verbose=0)

## Dropout

In [22]:
# BUILDING NEURAL NETWORK
model_dropout = Sequential()
model_dropout.add(Dense(500, input_dim=8, activation='sigmoid'))
model_dropout.add(Dropout(0.3,noise_shape=None, seed=None))
model_dropout.add(Dense(100, activation='sigmoid'))
model_dropout.add(Dropout(0.3,noise_shape=None, seed=None))
model_dropout.add(Dense(2, activation='softmax'))
model_dropout.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [41]:
model_dropout.fit(x_train,Y_train, epochs=1000, batch_size=70, validation_data=(x_test, Y_test),verbose=0)

## Recall,Precision,Accuracy,F1_score

In [42]:
# actual and predicted value for all the model above
y_l1=np.argmax(model_l1.predict(x_test),axis=1)
y_l2=np.argmax(model_l2.predict(x_test),axis=1)
y_es=np.argmax(model_es.predict(x_test),axis=1)
y_dropout=np.argmax(model_dropout.predict(x_test),axis=1)
y_pred=[y_l1,y_l2,y_es,y_dropout]
y_actual=np.argmax(Y_test, axis=1)

In [43]:
from sklearn.metrics import recall_score,precision_score,accuracy_score,f1_score
recall=[]
precision=[]
accuracy=[]
f1_score_=[]
for i in y_pred:
  r=recall_score(y_actual,i)
  recall.append(r)

  p=precision_score(y_actual,i)
  precision.append(p)

  a=accuracy_score(y_actual,i)
  accuracy.append(a)

  f=f1_score(y_actual, i, average='binary')
  f1_score_.append(f)

In [44]:
regularization=["L1 regularizer","L2 regularizer","Early Stop","Dropout"]
chr = ["Kernel=0.0001,Bias=0.0001,Activity=0.0001","Kernel=0.0001,Bias=0.0001,Activity=0.0001","Validation loss & patience=100","Dropout=0.3 in all hidden layer"]

In [45]:
result = pd.DataFrame.from_dict({"Type":regularization,"Charachterstics":chr,"Recall":recall,"Accuracy":accuracy,"Precision":precision,"F1 Score":f1_score_})

## Report

In [46]:
result

,Type,Charachterstics,Recall,Accuracy,Precision,F1 Score
0,L1 regularizer,"Kernel=0.0001,Bias=0.0001,Activity=0.0001",0.785047,0.740260,0.831683,0.807692
1,L2 regularizer,"Kernel=0.0001,Bias=0.0001,Activity=0.0001",0.850467,0.785714,0.842593,0.846512
2,Early Stop,Validation loss & patience=100,0.859813,0.779221,0.828829,0.844037
3,Dropout,Dropout=0.3 in all hidden layer,0.822430,0.785714,0.862745,0.842105


**Between L1 and L2 , L2 is performing better in terms of accuracy and F1_score**

**Between Early Stop and Dropout, Drop out is performing better in terms of accuracy and F1_score.**

**Comparing all models simultaneously, L2 Regularizer, Early stop and Dropout are performing equally well.**